**Fetch PG catalog**

In [ ]:
# If Google colab is being used

#from google.colab import drive
#drive.mount('/content/drive')

from pandas import read_csv

import numpy as np
import re
from datetime import datetime


# SETTINGS
pg_catalogue_loc = None
target_loc = None

Mounted at /content/drive


In [ ]:

gutenberg_catalog = read_csv(pg_catalogue_loc, dtype=str)
print(len(gutenberg_catalog))

def extend_catalog():
  gutenberg_catalog["year_google_api"] = np.nan
  gutenberg_catalog["year_google_html"] = np.nan
  gutenberg_catalog["year_loc"] = np.nan
  gutenberg_catalog["year_open_library"] = np.nan
  gutenberg_catalog["year_wikipedia"] = np.nan
  gutenberg_catalog["year_title"] = np.nan

#extend_catalog()


73570


Create some gold labels for sanity checks:

In [ ]:
ids = [0, 9, 10, 16, 53, 61, 75, 81, 83, 93]
ref_years = ["1776", "1611", "1865", "1830", "1904", "1912", "1884", "1819", "1818", "1912"]
titles = ["The Declaration of Independence of the United States of America",
          "The King James Version of the Bible",
          "Alice's Adventures in Wonderland",
          "The Book of Mormon: An Account Written by the Hand of Mormon, Upon Plates Taken from the Plates of Nephi",
          "The Marvelous Land of Oz",
          "A Princess of Mars",
          "Adventures of Huckleberry Finn",
          "Ivanhoe: A Romance",
          "Frankenstein; Or, The Modern Prometheus",
          "Alexander's Bridge"
          ]

# checking for correct mapping
for i in range(len(ids)):
  print(f"1: {titles[i]} \n2:{gutenberg_catalog.title[ids[i]]}")

1: The Declaration of Independence of the United States of America 
2:The Declaration of Independence of the United States of America
1: The King James Version of the Bible 
2:The King James Version of the Bible
1: Alice's Adventures in Wonderland 
2:Alice's Adventures in Wonderland
1: The Book of Mormon: An Account Written by the Hand of Mormon, Upon Plates Taken from the Plates of Nephi 
2:The Book of Mormon: An Account Written by the Hand of Mormon, Upon Plates Taken from the Plates of Nephi
1: The Marvelous Land of Oz 
2:The Marvelous Land of Oz
1: A Princess of Mars 
2:A Princess of Mars
1: Adventures of Huckleberry Finn 
2:Adventures of Huckleberry Finn
1: Ivanhoe: A Romance 
2:Ivanhoe: A Romance
1: Frankenstein; Or, The Modern Prometheus 
2:Frankenstein; Or, The Modern Prometheus
1: Alexander's Bridge 
2:Alexander's Bridge




---


**Google Books API: (not used)**

Metadata: https://support.google.com/books/partner/answer/3237055?hl=en

Query Design: https://developers.google.com/books/docs/v1/using?hl=de

In [ ]:
from urllib.request import urlopen
import json
import dateutil.parser as parser
import datetime as dt
import pandas as pd
from statistics import mode

In [ ]:
# Test mode behavior in case of ties
list1 = [2,3,4,5,6,2,3]
print(mode(list1))

2


In [ ]:
def getYearFromGoogleBooks(title, authors, birth = np.nan, death = np.nan):
  lowest_year = float('inf')

  # remove whitestrings from urls
  title = title.replace(" ","%20")
  if not pd.isna(authors):
    authors = authors.replace(" ","%20")

  # build google books API query URL
  queries = []
  if not pd.isna(authors):
    queries.append(f"https://www.googleapis.com/books/v1/volumes?q={title}%20{authors}")
    queries.append(f"https://www.googleapis.com/books/v1/volumes?q=intitle:{title}+inauthor:{authors}")
  queries.append(f"https://www.googleapis.com/books/v1/volumes?q=intitle:{title}")
  # send a request and get a JSON response
  for query in queries:
    resp = urlopen(query)

    # parse JSON into Python as a dictionary
    results = json.load(resp)
    items = results["items"]
    for book in items:
      if "publishedDate" in book["volumeInfo"].keys():
        year = parser.parse(book["volumeInfo"]["publishedDate"]).year
        if pd.isna(birth) or str(birth) < str(year) and (pd.isna(death) or str(year) <= str(death)):
          lowest_year = year if year < lowest_year else lowest_year

  # check if a possible publication year has been found
  return lowest_year if lowest_year != float('inf') else np.nan

In [ ]:
for authors, title, birth, death in zip(gutenberg_catalog.authors[ids], gutenberg_catalog.title[ids], gutenberg_catalog.first_author_birth_year[ids], gutenberg_catalog.first_author_death_year[ids]):
  print(title)
  print(getYearFromGoogleBooks(title, authors, birth, death))

print(ref_years)

The Declaration of Independence of the United States of America
nan
The King James Version of the Bible
2004
Alice's Adventures in Wonderland
1869
The Book of Mormon: An Account Written by the Hand of Mormon, Upon Plates Taken from the Plates of Nephi
nan
The Marvelous Land of Oz
1904
A Princess of Mars
1917
Adventures of Huckleberry Finn
nan
Ivanhoe: A Romance
1820
Frankenstein; Or, The Modern Prometheus
nan
Alexander's Bridge
1912
['1776', '1611', '1865', '1830', '1904', '1912', '1884', '1819', '1818', '1912']


In [ ]:
getYearFromGoogleBooks("The Declaration of Independence of the United States of America", "Jefferson, Thomas")

1834



---

**Scrape year from html of google search result (not used)**

Google will block requests after a few hundred

Custom Search Engine: https://programmablesearchengine.google.com/controlpanel/overview?cx=f5ae8ea749d7f40c2

Search Result format: https://developers.google.com/custom-search/v1/reference/rest/v1/Search#result



In [ ]:
from googleapiclient.discovery import build
def google_search(query, api_key, cse_id):
    service = build("customsearch", "v1", developerKey=api_key)
    res = service.cse().list(q=query, cx=cse_id).execute()
    return res

def google_search_year(title, author, author_birth, author_death, account = 0):
  api_keys = [TODO]
  cse_ids = [TODO]

  if pd.isna(author):
    query = f'In which year was "{title}" first published?'
  else:
    query = f'In which year was the book "{title}" written by {author} first published?'
  result = google_search(query, api_keys[account], cse_ids[account])

  pattern = r'(published|publication|created|wrote|written|release|released)[^\d]*(\d{4})'
  #pattern = r'[^\d]*(\d{4})'
  years = []
  # consists of the first 10 search results
  for item in result["items"]:
    text = item["snippet"]
    matches = re.findall(pattern, text, re.IGNORECASE)
    if matches:
        #print(f"{matches} \n {line}")
        if pd.isna(author_birth):
          years.extend([i[1] for i in matches])
        else:
          years.extend([i[1] for i in matches if i[1]> str(author_birth) and i[1]<= str(author_death)])
  # mode returns the most frequently occuring element in a list
  return mode(years) if len(years)>0 else None





In [ ]:
for authors, title, birth, death in zip(gutenberg_catalog.authors[ids], gutenberg_catalog.title[ids], gutenberg_catalog.first_author_birth_year[ids], gutenberg_catalog.first_author_death_year[ids]):
  print(google_search_year(title, authors, birth, death, 1))

print(ref_years)

1776
1611
1865
1830
1900
1912
1884
1819
1818
1912
['1776', '1611', '1865', '1830', '1904', '1912', '1884', '1819', '1818', '1912']


In [ ]:
import pandas as pd

import requests
from bs4 import BeautifulSoup
import re
import math

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36 Edg/124.0.0.0'}

In [ ]:
def build_query(title, author, mode = 'published'):

  query = 'https://www.google.de/search?q='
  if mode == 'published':
    if pd.isna(author):
      query += f'In which year was "{title}" first published?'
    else:
      query += f'In which year was the book "{title}" written by {author} first published?'
  elif mode == 'written':
    if pd.isna(author):
      query += f'In which year was "{title}" originally written?'
    else:
      query += f'When did {author} write the book "{title}"?'

  # remove whitestrings and quotation marks from urls
  query = query.replace(' ','%20')
  query = query.replace('"','%22')
  return query

In [ ]:
def fetch_year_from_google_search(query):
  #print(query)
  html = requests.get(query, headers=headers).text
  #print(html)
  soup = BeautifulSoup(html, 'lxml')
  extracts = []
  extracts.append(soup.find("div", class_="Z0LcW"))
  extracts.append(soup.find("b"))
  #print(f"1: {extracts[1]}")
  if str(extracts[1]) == "<b>About this page</b>":
    print(soup)
    raise Exception("You have been temporarily blocked by google")
  min_year = None
  for tag in extracts:
    if tag != None:
      year = re.search("\d{4}", str(tag))
      if year != None:
       year = year.group()
       min_year = year if min_year == None or year < min_year else min_year
  #print(f"2: {min_year}")
  return min_year

In [ ]:
i = 100
for authors, title in zip(gutenberg_catalog.authors, gutenberg_catalog.title):
    print(fetch_year_from_google_search(build_query(title, authors)))
    i+=1
#print(ref_years)

1776
None
None
None
1787
None
None
None
None
1611
1865
1871
1876
None
1851
1902
None
None
1855
1608
1871
1852
1845
1913
None
1608
1874
None
None
1611
None
1915
1850
1992
1895
1898
None
None
1987
None
None
1886
1886
1915
1908
1843
1909
None
None
2017
1915
None
None
1904
1900
None
None
1671
1637
1905
None
1912
None
1913
1993
None
None
1913
None
1906
1849
1916
1895
2004
None
1884
None
<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN">
<html>
<head><meta content="text/html; charset=utf-8" http-equiv="content-type"/><meta content="initial-scale=1" name="viewport"/><title>https://www.google.de/search?q=In%20which%20year%20was%20the%20book%20%22Tarzan%20of%20the%20Apes%22%20written%20by%20Burroughs,%20Edgar%20Rice%20first%20published?</title></head>
<body onload="e=document.getElementById('captcha');if(e){e.focus();} if(solveSimpleChallenge) {solveSimpleChallenge(,);}" style="font-family: arial, sans-serif; background-color: #fff; color: #000; padding:20px; font-size:18px; oversc

Exception: You have been temporarily blocked by google



---



**Library of Congress API**

doc: https://www.loc.gov/apis/json-and-yaml/requests/endpoints/

extended catalog maybe: https://www.loc.gov/apis/json-and-yaml/requests/endpoints/

In [ ]:
# urlopen throws error 403
import requests

In [ ]:
def build_query(author, title, author_birth=0, author_death=0):

  query = f'https://www.loc.gov/books/?fa=language:english&q={title} {author}&fo=json'

  # remove whitestrings and quotation marks from urls
  query = query.replace(' ','%20')
  query = query.replace('"','%22')
  #print(query)

  # send a request and get a JSON response
  #resp = urlopen(query)
  resp = requests.get(query)

  # parse JSON into Python as a dictionary
  json = resp.json()
  results = json["content"]["results"]

  years = []
  for item in results:
    if "date" in item and item["date"]> str(float(author_birth)+10) and item["date"] <= str(author_death):
      years.append(item["date"])
  print(years)


In [ ]:
for i in range(len(ids)):
  build_query(gutenberg_catalog.authors[ids[i]],
              gutenberg_catalog.title[ids[i]],
              gutenberg_catalog.first_author_birth_year[ids[i]],
              gutenberg_catalog.first_author_death_year[ids[i]])
  print(ref_years[i])

['1776-01-01', '1776-01-01', '1823', '1776', '1826']
1776
[]
1611
['1866', '1893', '1895', '1885', '1895', '1893', '1885']
1865
['1842', '1836']
1830
['1900', '1911', '1917', '1897', '1886', '1882', '1911', '1918', '1869', '1877', '1908', '1880', '1904']
1904
['1918', '1922', '1918', '1922', '1905', '1896', '1920', '1899', '1902', '1902', '1924', '1912']
1912
['1901', '1906', '1910', '1889', '1903', '1907', '1900']
1884
[]
1819
['1818', '1849']
1818
['1923', '1944', '1908', '1891-01-01', '1907-01-01', '1908-01-01', '1911', '1905', '1893', '1918', '1918']
1912




---



**Open Library API**

API docs: https://openlibrary.org/dev/docs/api/search



In [ ]:
import requests
from statistics import mode

def get_open_library_year(title, author, birth=np.nan, death=np.nan):
    first_publish_year = None
    url = f"https://openlibrary.org/search.json?q={title}+{author}"
    url = url.replace(' ','+')
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        docs = data['docs']
        years = []
        for doc in docs:
          year = doc.get('first_publish_year')
          if year != None and (pd.isna(birth) or float(birth) < year and (pd.isna(death) or year <= float(death))):
            years.append(year)
        return mode(years) if len(years)>0 else np.nan
    else:
        return np.nan


In [ ]:
for i in range(len(ids)):
  print(get_open_library_year(gutenberg_catalog.title[ids[i]],
              gutenberg_catalog.authors[ids[i]],
              gutenberg_catalog.first_author_birth_year[ids[i]],
              gutenberg_catalog.first_author_death_year[ids[i]],
              ))
  print(ref_years[i])

1776
1776
1537
1611
1889
1865
1830
1830
1904
1904
1917
1912
1876
1884
1800
1819
1818
1818
1912
1912


5/10

**Wikipedia API**

In [ ]:
!pip install wikipedia-api

In [ ]:
import wikipediaapi
from statistics import mode
import re
import pandas as pd

# Define a custom User-Agent
USER_AGENT = "BookYearFinderBot/1.0"

def get_wikipedia_page(title, user_agent):
    wiki_wiki = wikipediaapi.Wikipedia(USER_AGENT, 'en')
    page = wiki_wiki.page(title)
    return page

def extract_publication_year(page):
    if not page.exists():
        return None
    first_section = page.summary.split("\n")[0]
    content = page.text

    ## first check if any Year is mentioned in the first section of a wikipedia article
    pattern = r'[^\d]*(\d{4})'
    matches = re.findall(pattern, first_section, re.IGNORECASE)
    #print(f"matches: {matches}")
    if matches:
      #print(f"check: {matches}")
      # mode returns the most frequently occuring element in a list
      return mode(matches)

    # if not choose the Year that has been mentioned most frequently
    pattern = r'(published|created|wrote|written)[^\d]*(\d{4})'
    years = []
    lines = content.split("\n")
    for line in lines:
      matches = re.findall(pattern, line, re.IGNORECASE)
      if matches:
        #print(f"{matches} \n {line}")
        years.extend([i[1] for i in matches])
    # print(f"list: {years}")
    # mode returns the most frequently occuring element in a list
    return mode(years) if len(years)>0 else None

def wiki_publication_year(book_title, author_name, author_birth=np.nan, author_death=np.nan):
    # Construct a search query
    query = book_title
    page = get_wikipedia_page(query, USER_AGENT)
    try:
      page.exists()
    except:
      return np.nan

    if page.exists():
      publication_year = extract_publication_year(page)
      if publication_year is not None:
        if pd.isna(author_birth) or str(author_birth) < publication_year and (pd.isna(author_death) or publication_year <= str(author_death)):
          return publication_year
    return np.nan


In [ ]:
#for title in titles:
#  print(title)
print(wiki_publication_year(gutenberg_catalog.title[ids[0]],
                            gutenberg_catalog.authors[ids[0]]
              ))
print(ref_years[0])

None
1776
1776


In [ ]:
i = 0
for title, author, birth, death in zip(gutenberg_catalog.title[:10], gutenberg_catalog.authors[:10], gutenberg_catalog.first_author_birth_year[:10], gutenberg_catalog.first_author_death_year[:10]):
  i+=1
  print(f"{i}: {wiki_publication_year(title, author, birth, death)}, Title: {title}")

  if i == 10000:
        break


#1: 1776
#2: None
#3: 1961
#4: None
#5: 1789
#6: None
#7: 1646
#8: 1865
#9: None
#10: None

1: 1776, Title: The Declaration of Independence of the United States of America
2: None, Title: The United States Bill of Rights: The Ten Original Amendments to the Constitution of the United States
3: 1961, Title: John F. Kennedy's Inaugural Address
4: None, Title: Lincoln's Gettysburg Address: Given November 19, 1863 on the battlefield near Gettysburg, Pennsylvania, USA
5: 1789, Title: The United States Constitution
6: None, Title: Give Me Liberty or Give Me Death
7: 1646, Title: The Mayflower Compact
8: 1865, Title: Abraham Lincoln's Second Inaugural Address
9: None, Title: Abraham Lincoln's First Inaugural Address
10: None, Title: The King James Version of the Bible




---



**Get Year from Book Title**

In [ ]:
def year_from_title(title, author_birth, author_death, check = True):
    pattern = r'[^\d]*(\d{4})'
    matches = re.search(pattern, str(title), re.IGNORECASE)
    if matches:
      year = matches.group(1)
      if not check or pd.isna(author_birth) or str(author_birth) < year and str(author_death) >= year:
        return year
    return np.NaN




**Append df by years columns**

In [ ]:
def find_last_entry(df, col_name):
  last_entry = -1
  for i in range(len(df)):
    if not pd.isna(df[col_name][i]):
      last_entry = i
  return last_entry

In [ ]:
print(find_last_entry(gutenberg_catalog, "year_wikipedia"))
print(len(gutenberg_catalog))
print(find_last_entry(gutenberg_catalog, "year_google_html"))
print(find_last_entry(gutenberg_catalog, "year_title"))
print(find_last_entry(gutenberg_catalog, "year_open_library"))

73562
73570
958
73566
73569


In [ ]:
print(gutenberg_catalog.loc[35293])

id                                                                     35366
title                      Tom Clark and His Wife: Their Double Dreams, A...
authors                                            Randolph, Paschal Beverly
first_author_birth_year                                                 1825
first_author_death_year                                                 1874
translators                                                              NaN
type                                                                    Text
subjects                                             Rosicrucians -- Fiction
languages                                                                 en
formats                    text/html, text/html; charset=iso-8859-1, appl...
downloads                                                                 96
bookshelves                                                              NaN
copyright                                                              False

In [ ]:
# index = 35294 giving trouble

Page: Mad: A Story of Dust and Ashes (id: ??, ns: 0)


In [ ]:
from datetime import datetime

CHUNK_SIZE = 100
OFFSET = find_last_entry(gutenberg_catalog, "year_wikipedia") + 1 # In case fetching was stopped halfway through
print(f"starting at: {OFFSET}")
steps = (len(gutenberg_catalog)-OFFSET) // CHUNK_SIZE

for i in range(steps):
  gutenberg_catalog.loc[CHUNK_SIZE * i + OFFSET : CHUNK_SIZE * (i+1) + OFFSET, "year_wikipedia"] = gutenberg_catalog[CHUNK_SIZE * i + OFFSET : CHUNK_SIZE * (i+1) + OFFSET ].apply(lambda row: wiki_publication_year(row['title'], row["authors"], row["first_author_birth_year"], row["first_author_death_year"]), axis=1)
  print(f"{datetime.now()}: entries {i*CHUNK_SIZE + OFFSET}:{(i+1)*CHUNK_SIZE +  OFFSET} checked")
  gutenberg_catalog.to_csv(target_loc, index=False)
gutenberg_catalog.loc[CHUNK_SIZE * steps + OFFSET : , "year_wikipedia"] = gutenberg_catalog[CHUNK_SIZE * steps + OFFSET: len(gutenberg_catalog)].apply(lambda row: wiki_publication_year(row['title'], row["authors"], row["first_author_birth_year"], row["first_author_death_year"]), axis=1)


In [ ]:
find_last_entry(gutenberg_catalog, "year_google_html")

469

In [ ]:
# Script to query google as fast as possible without data loss while being blocked every now and then
import time

start = find_last_entry(gutenberg_catalog, "year_google_html")
print(f"Start: {start}")
missing = len(gutenberg_catalog) - start

delay = 1200
save_counter = 0

for i in range(missing):
  while True:
    try:
      gutenberg_catalog.loc[start+i+1, "year_google_html"] = fetch_year_from_google_search(build_query(gutenberg_catalog.title[start+i+1], gutenberg_catalog.authors[start+i+1]))
      save_counter +=1
      if save_counter == 20:
          print(f"saved until book {start+i+1}")
          gutenberg_catalog.to_csv(target_loc, index=False)
          save_counter = 0
      break
    except:
      gutenberg_catalog.to_csv(target_loc, index=False)
      delay *= 1.5
      print(f"delay has been set to: {delay}")
      time.sleep(delay)



Start: 786
saved until book 806
saved until book 826
saved until book 846
delay has been set to: 1800.0
delay has been set to: 2700.0
saved until book 866
saved until book 886
saved until book 906
delay has been set to: 4050.0
delay has been set to: 6075.0
saved until book 926
saved until book 946
delay has been set to: 9112.5


In [ ]:
# script to get year numbers from LoC API:

CHUNK_SIZE = 1000
OFFSET =  find_last_entry(gutenberg_catalog, "year_loc")+1 # In case fetching was stopped halfway through
steps = (len(gutenberg_catalog)-OFFSET) // CHUNK_SIZE

for i in range(steps):
  gutenberg_catalog.loc[CHUNK_SIZE * i + OFFSET : CHUNK_SIZE * (i+1) + OFFSET, "year_loc"] = gutenberg_catalog[CHUNK_SIZE * i + OFFSET : CHUNK_SIZE * (i+1) + OFFSET ].apply(lambda row: (row['title'], row["authors"], row["first_author_birth_year"], row["first_author_death_year"]), axis=1)
  print(f"entries {i*CHUNK_SIZE + OFFSET}:{(i+1)*CHUNK_SIZE +  OFFSET} checked")
  gutenberg_catalog.to_csv(target_loc, index=False)
gutenberg_catalog.loc[CHUNK_SIZE * steps + OFFSET: len(gutenberg_catalog), "year_wikipedia"] = gutenberg_catalog[CHUNK_SIZE * steps + OFFSET: len(gutenberg_catalog)].apply(lambda row: wiki_publication_year(row['title'], row["authors"], row["first_author_birth_year"], row["first_author_death_year"]), axis=1)


In [ ]:
  gutenberg_catalog["year_open_library"] = np.nan

In [ ]:


# script to fetch years from open library API

CHUNK_SIZE = 100
OFFSET = find_last_entry(gutenberg_catalog, "year_open_library")+1# In case fetching was stopped halfway through
print(f"starting at: {OFFSET}")
steps = (len(gutenberg_catalog)-OFFSET) // CHUNK_SIZE

for i in range(steps):
  gutenberg_catalog.loc[CHUNK_SIZE * i + OFFSET : CHUNK_SIZE * (i+1) + OFFSET, "year_open_library"] = gutenberg_catalog[CHUNK_SIZE * i + OFFSET : CHUNK_SIZE * (i+1) + OFFSET ].apply(lambda row: get_open_library_year(row['title'], row["authors"], row["first_author_birth_year"], row["first_author_death_year"]), axis=1)
  print(f"{datetime.now()}: entries {i*CHUNK_SIZE + OFFSET}:{(i+1)*CHUNK_SIZE +  OFFSET} checked")
  gutenberg_catalog.to_csv(target_loc, index=False)
gutenberg_catalog.loc[CHUNK_SIZE * steps + OFFSET: len(gutenberg_catalog), "year_open_library"] = gutenberg_catalog[CHUNK_SIZE * steps + OFFSET: len(gutenberg_catalog)].apply(lambda row: get_open_library_year(row['title'], row["authors"], row["first_author_birth_year"], row["first_author_death_year"]), axis=1)
gutenberg_catalog.to_csv(target_loc, index=False)


starting at: 73480


In [ ]:
  gutenberg_catalog.to_csv(target_loc, index=False)